<a href="https://colab.research.google.com/github/nczarli/f-jl221-2/blob/main/fine_tuning_DetrForSegmentation_on_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers timm

In [ ]:
!pip install -q pytorch-lightning

In [ ]:
!git clone https://github.com/nczarli/f-jl221-2.git

fatal: destination path 'f-jl221-2' already exists and is not an empty directory.


In [ ]:
%cd f-jl221-2/labelled_pits_and_not

/content/f-jl221-2/labelled_pits_and_not


In [ ]:
%ls

annotations/  images/


In [ ]:
import torchvision
import os

class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(self, annotation_path,image_path, feature_extractor, train=True):
        super(CocoDetection, self).__init__(image_path, annotation_path)
        self.feature_extractor = feature_extractor

    def __getitem__(self, idx):
        # read in PIL image and target in COCO format
        img, target = super(CocoDetection, self).__getitem__(idx)
        
        # preprocess image and target (converting target to DETR format, resizing + normalization of both image and target)
        image_id = self.ids[idx]
        target = {'image_id': image_id, 'annotations': target}
        encoding = self.feature_extractor(images=img, annotations=target, return_tensors="pt")
        pixel_values = encoding["pixel_values"].squeeze() # remove batch dimension
        target = encoding["labels"][0] # remove batch dimension

        return pixel_values, target

In [ ]:
from transformers import DetrFeatureExtractor

feature_extractor = DetrFeatureExtractor.from_pretrained("facebook/detr-resnet-50")

In [ ]:

annotation_path = './annotations/instances_default.json'
image_path = './images'

train_dataset = CocoDetection(annotation_path=annotation_path,image_path=image_path , feature_extractor=feature_extractor)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [ ]:
print("Number of training examples:", len(train_dataset))

Number of training examples: 10


In [ ]:
%ls

annotations/  images/


In [ ]:
import numpy as np
import os
from PIL import Image, ImageDraw

# based on https://github.com/woctezuma/finetune-detr/blob/master/finetune_detr.ipynb
image_ids = train_dataset.coco.getImgIds()
# let's pick a random image
image_id = image_ids[np.random.randint(0, len(image_ids))]
print('Image n°{}'.format(image_id))
image = train_dataset.coco.loadImgs(image_id)[0]
image = Image.open(os.path.join('/images', image['file_name']))

annotations = train_dataset.coco.imgToAnns[image_id]
draw = ImageDraw.Draw(image, "RGBA")

cats = train_dataset.coco.cats
id2label = {k: v['name'] for k,v in cats.items()}

for annotation in annotations:
  box = annotation['bbox']
  class_idx = annotation['category_id']
  x,y,w,h = tuple(box)
  draw.rectangle((x,y,x+w,y+h), outline='red', width=1)
  draw.text((x, y), id2label[class_idx], fill='white')

image

Image n°10


FileNotFoundError: ignored